In [2]:
#Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
from pprint import pprint


from config import gmaps_api_key


## State Coordinates

In [3]:
state_coords = pd.read_csv('state_coords.csv')
state_coords.head()

,State,latitude,longitude,Name
0,AK,63.588753,-154.493062,Alaska
1,AL,32.318231,-86.902298,Alabama
2,AR,35.201050,-91.831833,Arkansas
3,AZ,34.048928,-111.093731,Arizona
4,CA,36.778261,-119.417932,California


## Happiness Rating

In [4]:
happiness = pd.read_csv('happiness.csv')
happiness_coords = pd.merge(happiness, state_coords, on="Name")
happiness_coords.head()

,Name,Happiness Score,Happiness Rating,State,latitude,longitude
0,South Dakota,64.1,50,SD,43.969515,-99.901813
1,Vermont,64.1,49,VT,44.558803,-72.577841
2,Hawaii,63.4,48,HI,19.898682,-155.665857
3,Minnesota,63.1,47,MN,46.729553,-94.685900
4,North Dakota,63.1,46,ND,47.551493,-101.002012


## Parks Data

In [5]:
parks_key = ""
parks_url = f"https://api.nps.gov/api/v1/parks?limit=600&fields=addresses&key={parks_key}"
response = requests.get(parks_url)
parks_data = response.json()

park_names = []
park_city = []
park_state = []
park_zip = []

for park in parks_data['data']:
    try: 
        park_city.append(park['addresses'][0]['city'])
        park_names.append(park['fullName'])
        park_state.append(park['addresses'][0]['stateCode'])
        park_zip.append(park['addresses'][0]['postalCode'])
    except IndexError:
        print("x")

df = {
    'Park name': park_names, 'Park city': park_city, 'Park state': park_state, 'Park zip': park_zip,
}

park_df = pd.DataFrame(df)
park_df.dropna(inplace=True)

park_df.head()


x
x
x
x


,Park name,Park city,Park state,Park zip
0,Abraham Lincoln Birthplace National Historical...,Hodgenville,KY,42748
1,Acadia National Park,Bar Harbor,ME,04609
2,Adams National Historical Park,Quincy,MA,02169
3,African American Civil War Memorial,Washington,DC,20001
4,African Burial Ground National Monument,New York,NY,10007


In [11]:
# park_df.groupby("Park state").count().reset_index().rename(columns={'State': 'Count'})

park_count = park_df.groupby('Park state').count().reset_index()
park_count.columns = ["State", "Parks", 'City','Zip']
del park_count['City']
del park_count['Zip']

In [12]:
park_state_coords = pd.merge(park_count, state_coords, on="State")
park_state_coords.head()


,State,Parks,latitude,longitude,Name
0,AK,19,63.588753,-154.493062,Alaska
1,AL,7,32.318231,-86.902298,Alabama
2,AR,7,35.201050,-91.831833,Arkansas
3,AZ,21,34.048928,-111.093731,Arizona
4,CA,30,36.778261,-119.417932,California


## Dance Data

In [13]:
#Dance data courtesy of Jeff Kaufman
dance_df = pd.read_csv('dance.csv')
dance_df.head()

,Website,City,State,Weekday,Unnamed: 4,Role_name
0,http://www.thedancingbears.com/,Anchorage,AK,Fridays,20,
1,http://contraborealis.org/,Fairbanks,AK,Fridays and Saturdays,20,
2,http://www.juneaucontras.org/,Juneau,AK,Saturdays,5,
3,https://sites.google.com/site/bfootmad/,Birmingham,AL,Saturdays,12,
4,http://huntsvillecontra.dance,Huntsville,AL,Saturdays,27,


## Merge park & dance data

In [14]:
park_dance = pd.merge(park_state_coords, dance_count, on="State")
park_dance.head()

,State,Parks,latitude,longitude,Name,Dances
0,AK,19,63.588753,-154.493062,Alaska,3
1,AL,7,32.318231,-86.902298,Alabama,2
2,AR,7,35.201050,-91.831833,Arkansas,2
3,AZ,21,34.048928,-111.093731,Arizona,4
4,CA,30,36.778261,-119.417932,California,29


In [15]:
#Dance data courtesy of Jeff Kaufman

#Clean dance data
dance_count = dance_df.groupby('State').count().reset_index()
dance_count.columns = ["State", "Dances", 'Del','Weekday', 'Unnamed: 4', "Role_name"]

# del dance_count['State']
del dance_count['Unnamed: 4']
del dance_count['Role_name']
del dance_count['Del']
del dance_count['Weekday']

dance_count.head()

,State,Dances
0,AK,3
1,AL,2
2,AR,2
3,AZ,4
4,CA,29


## Education Data

In [16]:
education_df = pd.read_csv('education.csv')
education_coords = pd.merge(education_df, state_coords, on="Name")
education_coords.head()

,Name,Bachelors_pct,High_school_pct,State,latitude,longitude
0,Puerto Rico,12.289263,17.205459,PR,18.220833,-66.590149
1,Alabama,10.388658,17.151789,AL,32.318231,-86.902298
2,Alaska,11.956700,14.374090,AK,63.588753,-154.493062
3,Arizona,11.783280,13.357610,AZ,34.048928,-111.093731
4,Arkansas,9.425194,18.728458,AR,35.201050,-91.831833


## Public Transport Commuting 

In [17]:
public_transport = pd.read_csv('public_transport.csv')
public_transport_coords = pd.merge(public_transport, state_coords, on="Name")
public_transport_coords.head()

,Name,Public Transit Commute Pct,State,latitude,longitude
0,District of Columbia,47.3,DC,38.905985,-77.033418
1,New York,45.2,NY,43.299428,-74.217933
2,New Jersey,22.4,NJ,40.058324,-74.405661
3,Massachusetts,17.3,MA,42.407211,-71.382437
4,Illinois,16.6,IL,40.633125,-89.398528


## Income

In [18]:
income = pd.read_csv('income.csv')
income_coords = pd.merge(income, state_coords, on="Name")
income_coords.head()

,Name,Median Household Income,State,latitude,longitude
0,Puerto Rico,19775,PR,18.220833,-66.590149
1,Alabama,46472,AL,32.318231,-86.902298
2,Alaska,76114,AK,63.588753,-154.493062
3,Arizona,53510,AZ,34.048928,-111.093731
4,Arkansas,43813,AR,35.201050,-91.831833


## Building some maps

In [19]:
gmaps.configure(api_key={gmaps_api_key})
fig = gmaps.figure()

#Markers

#Parks
weights = park_state_coords['Parks'].astype(int)
locations = park_state_coords[["latitude", "longitude"]].astype(float)

#dances
dance_weights = park_dance['Dances'].astype(int)
dance_locations = park_dance[["latitude", "longitude"]].astype(float)

#Happiness
happiness_weights = happiness_coords['Happiness Rating'].astype(int)
happiness_locations = happiness_coords[["latitude", "longitude"]].astype(float)

#Education
education_weights = education_coords['Bachelors_pct'].astype(int)
education_locations = education_coords[["latitude", "longitude"]].astype(float)

#Public Transport
public_transport_weights = public_transport_coords['Public Transit Commute Pct'].astype(int)
public_transport_locations = public_transport_coords[["latitude", "longitude"]].astype(float)

#Income
income_weights = income_coords['Median Household Income'].astype(int)
income_locations = income_coords[["latitude", "longitude"]].astype(float)

## Heat layers
happiness_heat_layer = gmaps.heatmap_layer(happiness_locations, weights=happiness_weights,point_radius=1.5, dissipating=False)
# happiness_heat_layer.gradient = [(0,0,0,0), (0,255,255,.75), (0,0,255,.75),(0,0,128,.75)]

park_heat_layer = gmaps.heatmap_layer(locations, weights=weights,point_radius=1.5, dissipating=False)

dance_heat_layer = gmaps.heatmap_layer(dance_locations, weights=dance_weights,point_radius=1.5, dissipating=False)

education_heat_layer = gmaps.heatmap_layer(education_locations, weights=education_weights,point_radius=1.5, dissipating=False)

public_transport_heat_layer = gmaps.heatmap_layer(public_transport_locations, weights=public_transport_weights,point_radius=1.5, dissipating=False)

income_heat_layer = gmaps.heatmap_layer(income_locations, weights=income_weights,point_radius=1.5, dissipating=False)

fig.add_layer(park_heat_layer)
fig.add_layer(dance_heat_layer)
fig.add_layer(happiness_heat_layer)
fig.add_layer(education_heat_layer)
fig.add_layer(public_transport_heat_layer)
fig.add_layer(income_heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Census Ranking

In [31]:
ranking = pd.read_csv('census_ranking.csv')
ranking_coords = pd.merge(ranking, state_coords, on="Name", how="left")
ranking_coords

,Name,Total,State,latitude,longitude
0,Colorado,2,CO,39.550051,-105.782067
1,Massachusetts,1,MA,42.407211,-71.382437
2,Wyoming,3,WY,43.075968,-107.290284
3,Vermont,2,VT,44.558803,-72.577841
4,Alaska,1,AK,63.588753,-154.493062
5,Maryland,3,MD,39.045755,-76.641271
6,District of Columbia,8,DC,38.905985,-77.033418
7,California,6,CA,36.778261,-119.417932
8,New York,5,NY,43.299428,-74.217933
9,Massachusetts,2,MA,42.407211,-71.382437


In [34]:


#Ranking
ranking_weights = ranking_coords['Total'].astype(int)
ranking_locations = ranking_coords[["latitude", "longitude"]].astype(float)

marker_locations = [(39.550051,-105.782067),(42.407211,-71.382437),(43.075968,-107.290284),(44.558803,-72.577841),(63.588753,-154.493062),(39.045755,-76.641271),(38.905985,-77.033418),
(36.778261,-119.417932),(43.299428,-74.217933),(42.407211,-71.382437),(40.058324,-74.405661),(37.431573,-78.656894)]

ranking_heat_layer = gmaps.heatmap_layer(ranking_locations, weights=ranking_weights,point_radius=1.5, dissipating=False)

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)

fig.add_layer(ranking_heat_layer)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))